## Exercise - cudf.pandas - NYC Parking Violations

`cudf.pandas` is a tool that lets you use Pandas, while automatically accelerating operations on the GPU when possible. In this notebook we'll explore this capability by doing some data analysis using standard Pandas, and then accelerating it with `cudf.pandas`.

### Analysis using standard Pandas

In [ ]:
import pandas as pd

In [ ]:
%%time
# read 5 columns data:
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"],

)
df["Issue Date"] = df["Issue Date"].astype("datetime64[s]")

# view a random sample of 10 rows:
df.sample(10)

Each record in our dataset contains the state of registration of the offending vehicle, and the type of parking offence.


#### What is the most common type of offence by vehicles from different states?

Let's say we want to get the most common type of offence for vehicles registered in different states. We can do this in Pandas using a combination of value_counts and GroupBy.head:

In [ ]:
%%time
(df[["Registration State", "Violation Description"]]  # get only these two columns
 .value_counts()  # get the count of offences per state and per type of offence
 .groupby("Registration State")  # group by state
 .head(1)  # get the first row in each group (the type of offence with the largest count)
 .sort_index()  # sort by state name
 .reset_index()
)

The above code uses _method chaining_ to combine multiple operations into a single statement. 

#### Which vehicle body types are most frequently involved in parking violations?

We can also investigate which vehicle body types most commonly appear in parking violations

In [ ]:
%%time
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

#### How do parking violations vary across days of the week?

In [ ]:
%%time
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

### Exercise: using cudf.pandas

In this exercise, you'll repeat the analysis we did above using `cudf.pandas`. Fortunately, it needs no code changes at all.

1. Make note of the timings we obtained in the previous cells.
2. Restart this Jupyter notebook server (Kernel->Restart Kernel).
3. Before the line `import pandas`, insert the following line of code

```python
%load_ext cudf.pandas
```

4. Re-run the rest of the cells and note the new timings.

### Understanding Performance

`cudf.pandas` works by using the GPU for operations that are supported, and falling back to Pandas (CPU) when an operation is not supported.

The `line_profile` magic can help you figure out when 

In [ ]:
%%cudf.pandas.line_profile

small_df = pd.DataFrame({'a': ["0", "1", "2", "0", "1", "2"],
                         'b': ["x", "y", "z", "x", "y", "z"]})
small_df.min(axis=0)
small_df.min(axis=1)
counts = small_df.groupby("a").b.count()

### Exercise: Optimizing Pandas code for the GPU

`cudf.pandas` isn't completely magic. It's very possible to write code that will perform quite badly if you don't know what you're doing. Below is some code to extract just the records for violations in the month of March:

In [ ]:
%%time
def is_in_march(datetime):
    datetime = str(datetime)  # YYYY-MM-DD HH:MM:SS'
    date = datetime.split(" ")[0]
    year, month, day = date.split("-")
    return month == "03"

date = df["Issue Date"]
cond = date.apply(is_in_march)
df[cond].head()

The above snippet takes several seconds to complete - why do you think it's so slow? Rewrite the code above to do the same operation using a more efficient approach.

If successful, you should have code that takes a few milliseconds to run.